In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.models import Sequential, Model
from keras.layers import Dense, Flatten, BatchNormalization, Dropout, Conv2D, MaxPool2D, Softmax
from keras.datasets import mnist
from keras.optimizers import RMSprop
from keras.losses import SparseCategoricalCrossentropy
from keras.callbacks import ModelCheckpoint

In [89]:
print(tf.config.list_physical_devices())

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]


In [81]:
class ConvBlock(Model):
    def __init__(self, conv_count=2, filters=64, kernel_size=(3,3), pool_size=(2,2), padding='same', activation='relu'):
        super().__init__()
        self.conv = list()
        for i in range(conv_count):
            self.conv.append(Conv2D(filters=filters, kernel_size=kernel_size, padding=padding, activation=activation))
        self.bn = BatchNormalization()
        self.pool = MaxPool2D(pool_size=pool_size, padding=padding)
        self.dropout = Dropout(rate=0.2)
    
    def call(self, x):
        for layer in self.conv:
            x = layer(x)
        x = self.bn(x)
        x = self.pool(x)
        x = self.dropout(x)
        return x

In [82]:
class VGGNet(Model):
    def __init__(self):
        super().__init__()
        self.blocks = Sequential([
            ConvBlock(conv_count=2, filters=64),
            ConvBlock(conv_count=2, filters=128),
            ConvBlock(conv_count=4, filters=256),
            ConvBlock(conv_count=4, filters=512),
            ConvBlock(conv_count=4, filters=512),
            Flatten(),
            Dense(512),
            Dense(512),
            Dense(10, activation='softmax'),
        ])
    
    def call(self, x):
        return self.blocks(x)

In [83]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.astype(float)
x_train /= 255
x_test = x_test.astype(float)
x_test /= 255

x_train = x_train.reshape(-1, 28, 28, 1)
x_test = x_test.reshape(-1, 28, 28, 1)

In [84]:
batch_size = 128
epochs = 10

In [ ]:
net = VGGNet()
optimizer = RMSprop()
loss = SparseCategoricalCrossentropy(from_logits=True)
cpcallback = ModelCheckpoint(
    filepath='models/cp-{epoch}.ckpt',
    verbose=1,
    save_weights_only=True, 
    save_freq=batch_size
)

In [85]:
net.compile(optimizer=optimizer, 
            loss=loss,
            metrics=['accuracy'])

In [86]:
net(x_train[0].reshape(-1,28,28,1))

<tf.Tensor: shape=(1, 10), dtype=float32, numpy=
array([[0.09999923, 0.0999949 , 0.0999969 , 0.09999446, 0.10000442,
        0.10001107, 0.09999967, 0.0999971 , 0.09999875, 0.10000353]],
      dtype=float32)>

In [87]:
history = net.fit(
    x=x_train, 
    y=y_train, 
    batch_size=batch_size, 
    epochs=50, 
    validation_data=(x_test, y_test), 
    callbacks=[cpcallback]
)

Epoch 1/50


d:\anaconda3\lib\site-packages\keras\src\backend.py:5714: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


  7/469 [..............................] - ETA: 22:09 - loss: 174.4477

KeyboardInterrupt: 